In [1]:
import os
import sys
import time
import datetime
import argparse
import pickle
from skimage.data import imread
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torch.autograd import Variable
import torch.optim as optim
from PIL import Image

sys.path.append('/floyd/home/PyTorch-YOLOv3_lite')

In [2]:
%load_ext autoreload
%autoreload 2
from models import *
from utils.utils import *
from utils.datasets import *
from utils.parse_config import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
args = ['--image_folder','/floyd/input/airbus/train',
        '--box_file','/floyd/input/airbus/box_data.pickle',
        '--model_config_path','/floyd/home/PyTorch-YOLOv3_lite/config/classifier.cfg',
        '--weights_path','/floyd/input/weights/6.weights',
        '--checkpoint_dir','/floyd/home/output',
        '--img_size','64',
        '--epochs','30',
        '--n_cpu','4',
        '--use_cuda','True']
parser = argparse.ArgumentParser()
parser.add_argument('--epochs', type=int, default=30, help='number of epochs')
parser.add_argument('--image_folder', type=str, default='data/samples', help='path to dataset')
parser.add_argument('--box_file', type=str, default='data/samples', help='path to box data')
parser.add_argument('--batch_size', type=int, default=16, help='size of each image batch')
parser.add_argument('--model_config_path', type=str, default='config/yolov3.cfg', help='path to model config file')
parser.add_argument('--weights_path', type=str, default='weights/yolov3.weights', help='path to weights file')
parser.add_argument('--conf_thres', type=float, default=0.8, help='object confidence threshold')
parser.add_argument('--nms_thres', type=float, default=0.4, help='iou thresshold for non-maximum suppression')
parser.add_argument('--n_cpu', type=int, default=0, help='number of cpu threads to use during batch generation')
parser.add_argument('--img_size', type=int, default=416, help='size of each image dimension')
parser.add_argument('--checkpoint_interval', type=int, default=1, help='interval between saving model weights')
parser.add_argument('--checkpoint_dir', type=str, default='checkpoints', help='directory where model checkpoints are saved')
parser.add_argument('--use_cuda', type=bool, default=True, help='whether to use cuda if available')
opt = parser.parse_args(args)

In [4]:
cuda = torch.cuda.is_available() and opt.use_cuda

os.makedirs(opt.checkpoint_dir, exist_ok=True)

# Get hyper parameters
hyperparams     = parse_model_config(opt.model_config_path)[0]
learning_rate   = float(hyperparams['learning_rate'])
momentum        = float(hyperparams['momentum'])
decay           = float(hyperparams['decay'])
burn_in         = int(hyperparams['burn_in'])

# Initiate model
model = Darknet_Classifier(opt.model_config_path)
model.load_weights(opt.weights_path)
model.apply(weights_init_normal)

if cuda:
    model = model.cuda()

model.train();



In [15]:
class BoxDetections(Dataset):
    def __init__(self,imgdir,imgsize,data):
        super().__init__()
        self.imgdir = imgdir
        self.imgsize = imgsize
        self.box_image_ids = data['box_image_ids']
        self.L1 = data['L1']
        self.L2 = data['L2']
        self.R0 = data['C0'] # accidently switched when saved
        self.C0 = data['R0'] # accidently switched when saved
        self.theta = data['theta']
        self.all_image_ids = data['all_image_ids']
        self.num_ships = data['num_ships']
        self.total_ships = sum(self.num_ships)
        self.total_imgs = len(self.all_image_ids)
        self.max_objects = 20
    def __len__(self):
        return self.total_imgs
    def __getitem__(self,idx):
        img_path = os.path.join(self.imgdir,self.all_image_ids[idx])
        img = np.array(Image.open(img_path)) 
        # Channels-first
        img = np.transpose(img, (2, 0, 1))
        # As pytorch tensor
        img = torch.from_numpy(img).float()
        # Find boxes in img
        # Fill matrix
        labels = np.zeros((self.max_objects, 7)) 
        if self.num_ships[idx] != 0:
            box_inds = [i for i,x in enumerate(self.box_image_ids) if x is self.all_image_ids[idx]]
            for i,ind in enumerate(box_inds[:self.max_objects]):
                L1 = self.L1[ind]
                L2 = self.L2[ind]
                R0 = self.R0[ind]
                C0 = self.C0[ind]
                theta = self.theta[ind]
                # Calculate ratios from coordinates
                labels[i, 1] = R0 / self.imgsize
                labels[i, 2] = C0 / self.imgsize
                labels[i, 3] = L1 / self.imgsize
                labels[i, 4] = L2 / self.imgsize
                labels[i, 5] = np.sin(theta)
                labels[i, 6] = np.cos(theta)
        labels = torch.from_numpy(labels)
        return img_path, img, labels
    

In [16]:
# Get dataloader
with open(opt.box_file, 'rb') as f:
    data = pickle.load(f)
    
dataloader = torch.utils.data.DataLoader(
    BoxDetections(opt.image_folder,768,data),
    batch_size=opt.batch_size, shuffle=True, num_workers=opt.n_cpu, pin_memory=True)

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, dampening=0, weight_decay=decay)


In [17]:
bdtest = BoxDetections(opt.image_folder,768,data)

In [18]:
img_path, img, label = bdtest.__getitem__(2);

In [19]:
label

tensor([[ 0.0000,  0.9720,  0.1940,  0.0470,  0.0141, -0.8321,  0.5547],
        [ 0.0000,  0.9896,  0.1680,  0.0134,  0.0076, -0.8742,  0.4856],
        [ 0.0000,  0.9336,  0.1315,  0.0131,  0.0039,  0.9950,  0.0995],
        [ 0.0000,  0.8626,  0.2572,  0.0054,  0.0029,  0.9701,  0.2425],
        [ 0.0000,  0.8242,  0.2389,  0.0095,  0.0041, -0.2747,  0.9615],
        [ 0.0000,  0.9251,  0.1510,  0.0095,  0.0041,  0.9615,  0.2747],
        [ 0.0000,  0.9284,  0.1348,  0.0184,  0.0054,  0.9899,  0.1414],
        [ 0.0000,  0.9219,  0.1361,  0.0158,  0.0054,  0.9864,  0.1644],
        [ 0.0000,  0.9857,  0.2064,  0.0517,  0.0138, -0.7557,  0.6549],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.000